In [1]:
import os
import sys
import random
from scipy import signal
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# get filename_list
matplotlib.use('Agg')

In [21]:
import scipy.io
mat = scipy.io.loadmat('../data/preprocessed/resnet50.mat')
## load
X = mat["preprocessedData"]
Y = mat["meanlabels"]
ntrial = X.shape[0]

In [16]:
X.shape

(148, 3, 224, 224)

In [17]:
import torch
import torch.nn as nn
from scipy import stats
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torchvision.models import resnet50
from torchvision import datasets, models, transforms

In [67]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

class EmotionDataManager(Dataset):
    def __init__(self, data, label, transforms):
        self.df = data
        self.label = label.squeeze(1)
        self.transforms = transforms;
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        label = torch.tensor(self.label[index]-1).long()
        data = torch.tensor(self.df[index]).float()
#         print("hoge")
#         data = self.transforms(data)
        
        return data, label

In [5]:
def confusionMatrix

In [71]:
X.shape

(148, 3, 224, 224)

In [69]:
batch_size = 4
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)
description = "1129-resnet50-sgd-lr-0.001"

# datasize=range(ntrial)
datasize = range(ntrial)
epoches = 10

train_loss_value=np.empty((n_splits, epoches))
train_acc_value=np.empty((n_splits, epoches))
test_loss_value=np.empty((n_splits, epoches))
test_acc_value=np.empty((n_splits, epoches))
test_acc_each=np.zeros((n_splits, epoches, 5*5));

for fold, (train_index, test_index) in enumerate(kf.split(datasize)):
    train_x, train_y = X[train_index], Y[train_index]
    test_x, test_y = X[test_index], Y[test_index]
    traindataset = EmotionDataManager(train_x, train_y, data_transforms)
    testdataset = EmotionDataManager(test_x, test_y, data_transforms)
    trainloader = DataLoader(traindataset, batch_size,shuffle=True, num_workers=0)
    testloader = DataLoader(testdataset, batch_size,shuffle=True, num_workers=0)
    
    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs, 5)
    model_ft = model_ft.to(device)
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
    
    
    for epoch in range(epoches):
        print('epoch', epoch+1)
        print('-' * 10)

        sum_loss = 0.0
        sum_correct = 0
        sum_total = 0

        model_ft.train()
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer_ft.zero_grad()
            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)
#             l2_lambda = 0.0001
#             l2_norm = sum(p.pow(2.0).sum()
#                           for p in model.parameters())
#             loss = loss + l2_lambda * l2_norm
            loss.backward()
            optimizer_ft.step()

            sum_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            sum_total += labels.size(0)                 
            sum_correct += torch.sum(preds == labels)
        
        exp_lr_scheduler.step()
        
        print("train mean loss={}, accuracy={}".format(sum_loss*batch_size/len(trainloader.dataset), float(sum_correct/sum_total)))
        train_loss_value[fold, epoch] = sum_loss*batch_size/len(trainloader.dataset)
        train_acc_value[fold, epoch] = float(sum_correct/sum_total)

        sum_loss = 0.0
        sum_correct = 0
        sum_total = 0
        
#         label_each = np.zeros(5)
#         acc_each = np.zeros(5)

        model_ft.eval()
        for i, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer_ft.zero_grad()
            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)

            sum_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            sum_total += labels.size(0)                 
            sum_correct += torch.sum(preds == labels)
            
#             for l, lab in enumerate(labels):
#                 label_each[lab] += 1
#                 if (predicted[l] == lab):
#                     acc_each[lab] += 1

        print("test  mean loss={}, accuracy={}".format(sum_loss*batch_size/len(testloader.dataset), float(sum_correct/sum_total)))
        test_loss_value[fold, epoch] = sum_loss*batch_size/len(testloader.dataset)
        test_acc_value[fold, epoch] = float(sum_correct/sum_total)
#         for lab in range(5):
#             if  label_each[lab] > 0:
#                 test_acc_each[fold, epoch, lab] = acc_each[lab] / label_each[lab]


epoch 1
----------
train mean loss=1.661678564750542, accuracy=0.29661017656326294
test  mean loss=1.5286946773529053, accuracy=0.36666667461395264
epoch 2
----------
train mean loss=1.1535967891499148, accuracy=0.6101694703102112
test  mean loss=1.655027707417806, accuracy=0.36666667461395264
epoch 3
----------
train mean loss=0.6464260195271444, accuracy=0.7966101765632629
test  mean loss=1.8733041922251383, accuracy=0.36666667461395264
epoch 4
----------
train mean loss=0.5472180012424114, accuracy=0.8559321761131287
test  mean loss=2.21662784020106, accuracy=0.23333333432674408
epoch 5
----------
train mean loss=0.6113801063117335, accuracy=0.7542372941970825
test  mean loss=1.6233549992243448, accuracy=0.46666666865348816
epoch 6
----------
train mean loss=0.6338315176761756, accuracy=0.7881355881690979
test  mean loss=2.101747751235962, accuracy=0.2666666805744171
epoch 7
----------
train mean loss=0.5107162185644699, accuracy=0.8220338821411133
test  mean loss=2.0721645673116047

In [60]:
outputs.shape, labels.T.shape

(torch.Size([16, 5]), torch.Size([1, 16]))

In [66]:
criterion(outputs, labels.squeeze(1))

tensor(1.7235, grad_fn=<NllLossBackward0>)

In [65]:
labels.squeeze(1).shape

torch.Size([16])

In [70]:
for i in range(n_splits):
    matplotlib.use('Agg')

    fig = plt.figure(figsize=(30, 10))
    ax_right = fig.add_subplot(121)
    ax_left = fig.add_subplot(122)

    x = np.linspace(0, test_loss_value.shape[1], test_loss_value.shape[1], endpoint=False)
    ax_right.plot(x, train_loss_value[i, :], label="train")
    ax_right.plot(x, test_loss_value[i, :], label="test")
    ax_right.set_title("Loss at each epoch",fontsize=32)
    ax_right.set_xlabel("Epoch",fontsize=32)
    ax_right.set_ylabel("Loss",fontsize=32)
    ax_right.tick_params(axis='x', labelsize=24)
    ax_right.tick_params(axis='y', labelsize=24)
    ax_right.legend()


    ax_left.plot(x, train_acc_value[i, :], label="train")
    ax_left.plot(x, test_acc_value[i, :], label="test")
    ax_left.set_title("Accuracy at each epoch",fontsize=32)
    ax_left.set_xlabel("Epoch",fontsize=32)
    ax_left.set_ylabel("Accuracy rate",fontsize=32)
    ax_left.tick_params(axis='x', labelsize=24)
    ax_left.tick_params(axis='y', labelsize=24)
    ax_left.legend()

    plt.show()
    plt.savefig('../results/hiroo/cnn/%s-fold-%d.jpg' % (description, i))

/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_44940/2403428781.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_44940/2403428781.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_44940/2403428781.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_44940/2403428781.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/jh/nkpm2xxn2qdbzyljrfx984m40000gn/T/ipykernel_44940/2403428781.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
